In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([,.!?])", r" \1 ", s)
    s = re.sub(r"[^a-zA-Z,.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/MyDrive/Seq2Seq_Translation/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10486 sentence pairs
Counting words...
Counted words:
fra 4328
eng 2797
['elle passe a la tele ce soir .', 'she is appearing on tv tonight .']


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10486 sentence pairs
Counting words...
Counted words:
fra 4328
eng 2797
0m 37s (- 9m 16s) (5 6%) 1.5113
1m 13s (- 8m 31s) (10 12%) 0.6882
1m 48s (- 7m 50s) (15 18%) 0.3736
2m 24s (- 7m 13s) (20 25%) 0.2126
2m 59s (- 6m 34s) (25 31%) 0.1315
3m 52s (- 6m 27s) (30 37%) 0.0896
4m 33s (- 5m 52s) (35 43%) 0.0676
5m 11s (- 5m 11s) (40 50%) 0.0553
5m 47s (- 4m 30s) (45 56%) 0.0462
6m 22s (- 3m 49s) (50 62%) 0.0416
6m 57s (- 3m 9s) (55 68%) 0.0378
7m 35s (- 2m 31s) (60 75%) 0.0350
8m 11s (- 1m 53s) (65 81%) 0.0333
8m 46s (- 1m 15s) (70 87%) 0.0318
9m 22s (- 0m 37s) (75 93%) 0.0303
9m 56s (- 0m 0s) (80 100%) 0.0288


In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> vous etes grossieres .
= you re rude .
< you are rude to talk . <EOS>

> t es un gros nounours .
= you re a big softy .
< you re a big fan , too . <EOS>

> je n ai pas vraiment faim .
= i m not really hungry .
< i m not really hungry . <EOS>

> je vais juste faire une promenade .
= i am just going for a walk .
< i am just going for a walk . <EOS>

> il est nerveux d aller en amerique .
= he is anxious to go to america .
< he is anxious to go to america . <EOS>

> il est le meilleur ami de mon mari .
= he is my husband s best friend .
< he is my husband s best friend . <EOS>

> je n ai pas faim du tout .
= i m not hungry at all .
< i m not even a little hungry . <EOS>

> tu es un de ces dragueurs !
= you re such a flirt .
< you re such a flirt . <EOS>

> tu es une de ces menteuses !
= you are such a liar !
< you are such a liar ! <EOS>

> vous etes tous des racistes .
= you re all racists .
< you re all racists . <EOS>



In [ ]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    input_words = input_sentence.split(' ') + ['<EOS>']
    ax.set_xticks(range(len(input_words)))
    ax.set_yticks(range(len(output_words)))

    ax.set_xticklabels(input_words, rotation=90)
    ax.set_yticklabels(output_words)

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])
    plt.show()


evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')

input = il n est pas aussi grand que son pere
output = he is not as tall as his father . <EOS>
input = je suis trop fatigue pour conduire
output = i m too tired to drive . <EOS>
input = je suis desole si c est une question idiote
output = i m sorry so a night problem . <EOS>
input = je suis reellement fiere de vous
output = i m really proud of you . <EOS>


# Exercises
- Try with a different dataset

    - Another language pair

    - Human → Machine (e.g. IOT commands)

    - Chat → Response

    - Question → Answer

- Replace the embeddings with pretrained word embeddings such as word2vec or GloVe

- Try with more layers, more hidden units, and more sentences. Compare the training time and results.

- If you use a translation file where pairs have two of the same phrase (I am test \t I am test), you can use this as an autoencoder. Try this:

    - Train as an autoencoder

    - Save only the Encoder network

    - Train a new Decoder for translation from there

In [ ]:
# MIN_LENGTH = 3
# MAX_LENGTH = 25

# def filterPairs(pairs):
#     filtered_pairs = []
#     for pair in pairs:
#         if len(pair[0]) >= MIN_LENGTH and len(pair[0]) <= MAX_LENGTH \
#             and len(pair[1]) >= MIN_LENGTH and len(pair[1]) <= MAX_LENGTH:
#                 filtered_pairs.append(pair)
#     return filtered_pairs

In [ ]:
# NOT DOING THIS, BUT YOU CAN TRY
# MAX_LENGTH = 10

# eng_prefixes = (
#     "i am ", "i m ",
#     "he is", "he s ",
#     "she is", "she s ",
#     "you are", "you re ",
#     "we are", "we re ",
#     "they are", "they re "
# )

# def filterPair(p):
#     return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

# def filterPairs(pairs):
#     return [pair for pair in pairs if filterPair(pair)]

In [ ]:
# a = ['I am sick.', 'میں بیمار ہوں۔']
# a = [ai.lower() for ai in a]
# print(a)

In [ ]:
# filterPair(a)

In [ ]:
# a[0], a[1]

In [ ]:
# len(a[0].split(' ')) < MAX_LENGTH, len(a[1].split(' ')) < MAX_LENGTH, a[1].startswith(eng_prefixes)

In [ ]:
# def prepareData(lang1, lang2, reverse=False):
#     input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
#     print("Read %d sentence pairs" % len(pairs))

#     pairs = filterPairs(pairs)
#     # print(pairs)
#     print("Filtered to %d pairs" % len(pairs))

#     print("Indexing words...")
#     for pair in pairs:
#         input_lang.addSentence(pair[0])
#         output_lang.addSentence(pair[1])

#     print('Indexed %d words in input language, %d words in output' % (input_lang.n_words, output_lang.n_words))
#     return input_lang, output_lang, pairs

# input_lang, output_lang, pairs = prepareData('eng', 'fra', False)

Reading lines...
Read 135842 sentence pairs
Filtered to 25717 pairs
Indexing words...
Indexed 4346 words in input language, 7002 words in output


In [ ]:
# print(random.choice(pairs))

['it is too expensive .', 'c est trop cher .']


In [ ]:
# MIN_COUNT = 5

# input_lang.trim(MIN_COUNT)
# output_lang.trim(MIN_COUNT)

keep_words 1530 / 4343 = 0.3523
keep_words 1718 / 6999 = 0.2455


In [ ]:
# keep_pairs = []

# for pair in pairs:
#     input_sentence = pair[0]
#     output_sentence = pair[1]
#     keep_input = True
#     keep_output = True

#     for word in input_sentence.split(' '):
#         if word not in input_lang.word2index:
#             keep_input = False
#             break

#     for word in output_sentence.split(' '):
#         if word not in output_lang.word2index:
#             keep_output = False
#             break

#     # Remove if pair doesn't match input and output conditions
#     if keep_input and keep_output:
#         keep_pairs.append(pair)

# print("Trimmed from %d pairs to %d, %.4f of total" % (len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
# pairs = keep_pairs

Trimmed from 25717 pairs to 15909, 0.6186 of total


In [ ]:
# # Return a list of indexes, one for each word in the sentence, plus EOS
# def indexesFromSentence(lang, sentence):
#     return [lang.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [ ]:
# # Pad a with the PAD symbol
# def padSeq(seq, max_length):
#     seq += [PAD_token for i in range(max_length - len(seq))]
#     return seq

In [ ]:
# def random_batch(batch_size):
#     input_seqs = []
#     target_seqs = []

#     # Choose random pairs
#     for i in range(batch_size):
#         pair = random.choice(pairs)
#         input_seqs.append(indexesFromSentence(input_lang, pair[0]))
#         target_seqs.append(indexesFromSentence(output_lang, pair[1]))

#     # Zip into pairs, sort by length (descending), unzip
#     seq_pairs = sorted(zip(input_seqs, target_seqs), key=lambda p: len(p[0]), reverse=True)
#     input_seqs, target_seqs = zip(*seq_pairs)

#     # For input and target sequences, get array of lengths and pad with 0s to max length
#     input_lengths = [len(s) for s in input_seqs]
#     input_padded = [padSeq(s, max(input_lengths)) for s in input_seqs]
#     target_lengths = [len(s) for s in target_seqs]
#     target_padded = [padSeq(s, max(target_lengths)) for s in target_seqs]

#     # Turn padded arrays into (batch_size x max_len) tensors, transpose into (max_len x batch_size)
#     input_var = Variable(torch.LongTensor(input_padded)).transpose(0, 1)
#     target_var = Variable(torch.LongTensor(target_padded)).transpose(0, 1)

#     if USE_CUDA:
#         input_var = input_var.cuda()
#         target_var = target_var.cuda()

#     return input_var, input_lengths, target_var, target_lengths

In [ ]:
# random_batch(2)

(tensor([[ 12,  12],
         [102, 211],
         [186,  91],
         [  3, 602],
         [  4,   4],
         [  2,   2]]),
 [6, 6],
 tensor([[112,  20],
         [209, 337],
         [325, 108],
         [196, 849],
         [ 58,  14],
         [245,   2],
         [ 14,   0],
         [  2,   0]]),
 [8, 6])

In [ ]:
# class EncoderRNN(nn.Module):
#     def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
#         super(EncoderRNN, self).__init__()

#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.n_layers = n_layers
#         self.dropout = dropout

#         self.embedding = nn.Embedding(input_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)

#     def forward(self, input_seqs, input_lengths, hidden=None):
#         # Note: we run this all at once (over multiple batches of multiple sequences)
#         embedded = self.embedding(input_seqs)
#         packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
#         outputs, hidden = self.gru(packed, hidden)
#         outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
#         outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
#         return outputs, hidden

In [ ]:
# class Attn(nn.Module):
#     def __init__(self, method, hidden_size):
#         super(Attn, self).__init__()
#         self.method = method
#         if self.method not in ['dot', 'general', 'concat']:
#             raise ValueError(self.method, "is not an appropriate attention method.")
#         self.hidden_size = hidden_size
#         if self.method == 'general':
#             self.attn = nn.Linear(self.hidden_size, hidden_size)
#         elif self.method == 'concat':
#             self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
#             self.v = nn.Parameter(torch.FloatTensor(hidden_size))

#     def dot_score(self, hidden, encoder_output):
#         return torch.sum(hidden * encoder_output, dim=2)

#     def general_score(self, hidden, encoder_output):
#         energy = self.attn(encoder_output)
#         return torch.sum(hidden * energy, dim=2)

#     def concat_score(self, hidden, encoder_output):
#         energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
#         return torch.sum(self.v * energy, dim=2)

#     def forward(self, hidden, encoder_outputs):
#         # Calculate the attention weights (energies) based on the given method
#         if self.method == 'general':
#             attn_energies = self.general_score(hidden, encoder_outputs)
#         elif self.method == 'concat':
#             attn_energies = self.concat_score(hidden, encoder_outputs)
#         elif self.method == 'dot':
#             attn_energies = self.dot_score(hidden, encoder_outputs)

#         # Transpose max_length and batch_size dimensions
#         attn_energies = attn_energies.t()

#         # Return the softmax normalized probability scores (with added dimension)
#         return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
# class BahdanauAttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
#         super(BahdanauAttnDecoderRNN, self).__init__()

#         # Define parameters
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.n_layers = n_layers
#         self.dropout_p = dropout_p
#         self.max_length = max_length

#         # Define layers
#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.dropout = nn.Dropout(dropout_p)
#         self.attn = Attn('concat', hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout_p)
#         self.out = nn.Linear(hidden_size, output_size)

#     def forward(self, word_input, last_hidden, encoder_outputs):
#         # Note: we run this one step at a time
#         # TODO: FIX BATCHING

#         # Get the embedding of the current input word (last output word)
#         word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N
#         word_embedded = self.dropout(word_embedded)

#         # Calculate attention weights and apply to encoder outputs
#         attn_weights = self.attn(last_hidden[-1], encoder_outputs)
#         context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N
#         context = context.transpose(0, 1) # 1 x B x N

#         # Combine embedded input word and attended context, run through RNN
#         rnn_input = torch.cat((word_embedded, context), 2)
#         output, hidden = self.gru(rnn_input, last_hidden)

#         # Final output layer
#         output = output.squeeze(0) # B x N
#         output = F.log_softmax(self.out(torch.cat((output, context), 1)))

#         # Return final output, hidden state, and attention weights (for visualization)
#         return output, hidden, attn_weights

In [ ]:
# class LuongAttnDecoderRNN(nn.Module):
#     def __init__(self, attn_model, hidden_size, output_size, n_layers=1, dropout=0.1):
#         super(LuongAttnDecoderRNN, self).__init__()

#         # Keep for reference
#         self.attn_model = attn_model
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.n_layers = n_layers
#         self.dropout = dropout

#         # Define layers
#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.embedding_dropout = nn.Dropout(dropout)
#         self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=dropout)
#         self.concat = nn.Linear(hidden_size * 2, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)

#         # Choose attention model
#         if attn_model != 'none':
#             self.attn = Attn(attn_model, hidden_size)

#     def forward(self, input_seq, last_hidden, encoder_outputs):
#         # Note: we run this one step at a time

#         # Get the embedding of the current input word (last output word)
#         batch_size = input_seq.size(0)
#         embedded = self.embedding(input_seq)
#         embedded = self.embedding_dropout(embedded)
#         embedded = embedded.view(1, batch_size, self.hidden_size) # S=1 x B x N

#         # Get current hidden state from input word and last hidden state
#         rnn_output, hidden = self.gru(embedded, last_hidden)

#         # Calculate attention from current RNN state and all encoder outputs;
#         # apply to encoder outputs to get weighted average
#         attn_weights = self.attn(rnn_output, encoder_outputs)
#         context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x S=1 x N

#         # Attentional vector using the RNN hidden state and context vector
#         # concatenated together (Luong eq. 5)
#         rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
#         context = context.squeeze(1)       # B x S=1 x N -> B x N
#         concat_input = torch.cat((rnn_output, context), 1)
#         concat_output = F.tanh(self.concat(concat_input))

#         # Finally predict next token (Luong eq. 6, without softmax)
#         output = self.out(concat_output)

#         # Return final output, hidden state, and attention weights (for visualization)
#         return output, hidden, attn_weights

In [ ]:
# small_batch_size = 3
# input_batches, input_lengths, target_batches, target_lengths = random_batch(small_batch_size)

# print('input_batches', input_batches.size()) # (max_len x batch_size)
# print('target_batches', target_batches.size()) # (max_len x batch_size)

input_batches torch.Size([7, 3])
target_batches torch.Size([8, 3])


In [ ]:
# input_batches

tensor([[  12,  146,   22],
        [1037,  191,   70],
        [ 186,  872,  324],
        [ 203,  186,  272],
        [1155,   94,    4],
        [   4,    4,    2],
        [   2,    2,    0]])

In [ ]:
# small_hidden_size = 8
# small_n_layers = 2

# encoder_test = EncoderRNN(input_lang.n_words, small_hidden_size, small_n_layers)
# decoder_test = LuongAttnDecoderRNN('general', small_hidden_size, output_lang.n_words, small_n_layers)

# if USE_CUDA:
#     encoder_test.cuda()
#     decoder_test.cuda()

In [ ]:
# encoder_outputs, encoder_hidden = encoder_test(input_batches, input_lengths, None)

# print('encoder_outputs', encoder_outputs.size()) # max_len x batch_size x hidden_size
# print('encoder_hidden', encoder_hidden.size()) # n_layers * 2 x batch_size x hidden_size

encoder_outputs torch.Size([7, 3, 8])
encoder_hidden torch.Size([4, 3, 8])


In [ ]:
# max_target_length = max(target_lengths)

# # Prepare decoder input and outputs
# decoder_input = Variable(torch.LongTensor([SOS_token] * small_batch_size))
# decoder_hidden = encoder_hidden[:decoder_test.n_layers] # Use last (forward) hidden state from encoder
# all_decoder_outputs = Variable(torch.zeros(max_target_length, small_batch_size, decoder_test.output_size))

# if USE_CUDA:
#     all_decoder_outputs = all_decoder_outputs.cuda()
#     decoder_input = decoder_input.cuda()

# # Run through decoder one time step at a time
# for t in range(max_target_length):
#     decoder_output, decoder_hidden, decoder_attn = decoder_test(
#         decoder_input, decoder_hidden, encoder_outputs
#     )
#     all_decoder_outputs[t] = decoder_output # Store this step's outputs
#     decoder_input = target_batches[t] # Next input is current target

# # Test masked cross entropy loss
# loss = F.cross_entropy(
#     all_decoder_outputs.transpose(0, 1).contiguous(),
#     target_batches.transpose(0, 1).contiguous(),
#     target_lengths
# )
# print('loss', loss.data[0])